<a href="https://colab.research.google.com/github/Tipicomm/DPNC/blob/main/RequetePropDataMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests

# URL de la ressource
url = "https://www.data.gouv.fr/api/1/organizations/ministere-de-la-culture-et-de-la-communication/datasets-resources.csv"
fichier_telecharge = "datasets-resources.csv"

# Télécharger le fichier et l’enregistrer localement
response = requests.get(url)
with open(fichier_telecharge, "wb") as f:
    f.write(response.content)

print("📥 Fichier téléchargé et enregistré localement sous 'datasets-resources.csv'.")

# Lire le CSV
df = pd.read_csv(fichier_telecharge, sep=";")

# Filtrer uniquement les ressources CSV (insensible à la casse)
df_csv = df[df["format"].str.lower() == "csv"]

# Ne conserver que les colonnes souhaitées
colonnes_souhaitees = ["dataset.id", "dataset.title", "id"]
df_csv = df_csv[colonnes_souhaitees]

# Enregistrer dans un nouveau fichier
df_csv.to_csv("ressources_csv_culture.csv", index=False)

print("✅ Fichier 'ressources_csv_culture.csv' généré avec succès (colonnes filtrées).")


📥 Fichier téléchargé et enregistré localement sous 'datasets-resources.csv'.
✅ Fichier 'ressources_csv_culture.csv' généré avec succès (colonnes filtrées).


In [ ]:
import pandas as pd
import requests
import yaml
from tqdm import tqdm

# Charger les 50 premières ressources
df_csv = pd.read_csv("ressources_csv_culture.csv").head(200)

# Stocker les résultats sous forme de lignes individuelles
rows = []

for _, row in tqdm(df_csv.iterrows(), total=len(df_csv), desc="🔍 Extraction propriétés"):
    resource_id = row["id"]
    dataset_id = row["dataset.id"]
    dataset_title = row["dataset.title"]

    url = f"https://tabular-api.data.gouv.fr/api/resources/{resource_id}/swagger/"

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            try:
                data = yaml.safe_load(response.text)
                props = data.get("components", {}).get("schemas", {}).get("Resource", {}).get("properties", {})

                if props:
                    for prop_name, prop_info in props.items():
                        rows.append({
                            "id": resource_id,
                            "dataset.id": dataset_id,
                            "dataset.title": dataset_title,
                            "property_name": prop_name,
                            "property_type": prop_info.get("type", "unknown")
                        })
                else:
                    rows.append({
                        "id": resource_id,
                        "dataset.id": dataset_id,
                        "dataset.title": dataset_title,
                        "property_name": "⚠️ Aucune colonne trouvée",
                        "property_type": ""
                    })
            except yaml.YAMLError as e:
                rows.append({
                    "id": resource_id,
                    "dataset.id": dataset_id,
                    "dataset.title": dataset_title,
                    "property_name": f"❌ Erreur YAML : {str(e)}",
                    "property_type": ""
                })
        else:
            rows.append({
                "id": resource_id,
                "dataset.id": dataset_id,
                "dataset.title": dataset_title,
                "property_name": f"❌ HTTP {response.status_code}",
                "property_type": ""
            })
    except Exception as e:
        rows.append({
            "id": resource_id,
            "dataset.id": dataset_id,
            "dataset.title": dataset_title,
            "property_name": f"❌ Exception : {str(e)}",
            "property_type": ""
        })

# Convertir en DataFrame et sauvegarder
df_properties = pd.DataFrame(rows)
df_properties.to_csv("ressources_csv_properties.csv", index=False)

print("✅ Fichier 'ressources_csv_properties.csv' généré (format long par propriété).")


🔍 Extraction propriétés: 100%|██████████| 190/190 [01:33<00:00,  2.04it/s]


✅ Fichier 'ressources_csv_properties.csv' généré (format long par propriété).
